import numpy as np: This imports the NumPy library, which is a powerful library for numerical computations in Python.

np.random.seed(10): This sets the random seed for NumPy to ensure reproducibility of the simulation results.

class FastFoodRestaurant:: This defines a class called FastFoodRestaurant that represents a fast food restaurant simulation.

def __init__(self, num_servers):: This is the constructor of the FastFoodRestaurant class, which is called when an instance of the class is created. It takes the number of servers as an argument and initializes various attributes, such as num_servers, interarrivals, service_times, clocks, next_arrivals, next_departures, num_in_queue, times_of_arrival_in_queue, service_times_in_queue, total_delays, num_of_delays, customers_served, and server_statuses, with appropriate initial values.

def start(self):: This method simulates the events in the fast food restaurant until 50 customers are served. It repeatedly calls the simulate_next_event() method to simulate the next event.

def simulate_next_event(self):: This method simulates the next event in the fast food restaurant. It determines the next event (either an arrival or a departure) based on the minimum of the next arrival times and next departure times, and updates the simulation clock accordingly. It then calls the appropriate methods arrival() or departure() based on the event type, and updates the simulation state and attributes accordingly. Finally, it prints the current status of the simulation, such as server statuses, number of customers in the queue, total delay, next arrival time, and next departure times.

def arrival(self):: This method simulates an arrival event in the fast food restaurant. It updates the next arrival time, and if there is an available server, it updates the next departure time for that server. Otherwise, it adds the customer to the queue and records their arrival time and service time in the queue.

def departure(self, server_idx):: This method simulates a departure event from a specific server in the fast food restaurant. It updates the customer served count, releases the server by updating its status, and if there are customers in the queue, it dequeues the next customer and updates the next departure time for the server based on the service time of the dequeued customer. Otherwise, it sets the next departure time for the server to infinity.

The code at the end creates an instance of the FastFoodRestaurant class with a specific number of servers, and calls the start() method to begin the simulation.

In [1]:
import math
import random
import numpy as np

np.random.seed(10)

class FastFoodRestaurant:
    def __init__(self, num_servers):
        if num_servers <= 0:
            raise ValueError("Number of servers must be positive")
        self.num_servers = num_servers
        self.interarrivals = list(np.random.exponential(scale=5, size=100))
        self.service_times = list(np.random.uniform(low=1, high=4, size=100))
        self.clocks = [0.0] * (num_servers + 1)
        self.next_arrivals = [self.interarrivals.pop(0)]
        self.next_departures = [float('inf')] * (num_servers + 1)
        self.num_in_queue = 0
        self.times_of_arrival_in_queue = []
        self.service_times_in_queue = []
        self.total_delays = 0.0
        self.num_of_delays = 0
        self.customers_served = 0
        self.server_statuses = [0] * num_servers
        self.last_event_time = 0.0

    def start(self):
        while self.customers_served < 25:
            self.simulate_next_event()

    def simulate_next_event(self):
        min_clock = min(self.next_arrivals + self.next_departures)
        event = np.argmin(self.next_arrivals + self.next_departures)

        self.clock = min_clock
        self.event = event

        if event == 0:
            self.arrival()
            print("Arrival at Clock: {:.3f}`".format(self.clock))
        else:
            server_idx = event - 1
            self.departure(server_idx)
            print("Departure from Server " + str(server_idx) + " at Clock: {:.3f}".format(self.clock))
     

        print("Queue Status:")
        print("Number of Servers: " + str(self.num_servers))
        print("Server Statuses: " + str(self.server_statuses))
        print("Number of Customers in Queue: " + str(self.num_in_queue))
        print("Times of Arrival in Queue: " + str([round(t, 3) for t in self.times_of_arrival_in_queue]))
        print("Service Times in Queue: " + str([round(t, 3) for t in self.service_times_in_queue]))
        print("Total Delay: " + str(round(self.total_delays, 3)))
        print("Next Arrival Time: " + str(round(self.next_arrivals[0], 3)))
        print("Next Departure Times: " + str([round(t, 3) for t in self.next_departures[1:]]))
        print(" ")

        self.last_event_time = self.clock

    def arrival(self):
        self.next_arrivals[0] += self.interarrivals.pop(0)
        if sum(self.server_statuses) < self.num_servers:
            server_idx = self.server_statuses.index(0)
            self.server_statuses[server_idx] = 1 #status update to busy
            self.next_departures[server_idx + 1] = self.clock + self.service_times.pop(0) #current time + service time
        else:
            self.num_in_queue += 1
            self.times_of_arrival_in_queue.append(self.clock)
            self.service_times_in_queue.append(self.service_times.pop(0))
        self.next_arrivals.sort()  # Sort next_arrivals to maintain sorted order


    def departure(self, server_idx):
        self.customers_served += 1
        if server_idx >= 0 and server_idx < self.num_servers:
            self.server_statuses[server_idx] = 0 #update server status to idle
        else:
            return
        if self.num_in_queue > 0:
            self.num_in_queue -= 1
            self.total_delays += max(0, self.clock - self.times_of_arrival_in_queue.pop(0)) # calculate the delay for the departing customer as the difference between the current time and the arrival time of the customer at the front of the queue. 
            self.num_of_delays += 1
            self.next_departures[server_idx + 1] = self.clock + self.service_times_in_queue.pop(0)
        else:
            self.next_departures[server_idx + 1] = float('inf') #if there are no customers in the queue, set the departure time of the current server to infinity, indicating that there are no customers waiting to be served by that server.

        # Update the departure time of the current server
        self.next_departures[server_idx] = float('inf')
         
    def average_delay(self):
      if self.num_of_delays == 0:
          return 0.0
      else:
          return self.total_delays / self.num_of_delays

s = FastFoodRestaurant(num_servers=3)
s.start()
print("Average Delay: {:.3f}".format(s.average_delay()))




Arrival at Clock: 7.377`
Queue Status:
Number of Servers: 3
Server Statuses: [1, 0, 0]
Number of Customers in Queue: 0
Times of Arrival in Queue: []
Service Times in Queue: []
Total Delay: 0.0
Next Arrival Time: 7.482
Next Departure Times: [10.112, inf, inf]
 
Arrival at Clock: 7.482`
Queue Status:
Number of Servers: 3
Server Statuses: [1, 1, 0]
Number of Customers in Queue: 0
Times of Arrival in Queue: []
Service Times in Queue: []
Total Delay: 0.0
Next Arrival Time: 12.503
Next Departure Times: [10.112, 11.044, inf]
 
Departure from Server 1 at Clock: 10.112
Queue Status:
Number of Servers: 3
Server Statuses: [1, 0, 0]
Number of Customers in Queue: 0
Times of Arrival in Queue: []
Service Times in Queue: []
Total Delay: 0.0
Next Arrival Time: 12.503
Next Departure Times: [inf, inf, inf]
 
Arrival at Clock: 12.503`
Queue Status:
Number of Servers: 3
Server Statuses: [1, 1, 0]
Number of Customers in Queue: 0
Times of Arrival in Queue: []
Service Times in Queue: []
Total Delay: 0.0
Next 